In [1]:
from PIL import Image
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [13]:
folders = ['TM_Chassis_MULT', 'TM_Chassis_Single', 'TM-A_Chassis_MULT', 'TM-A_Chassis_Single']
PCB_num = 0

folder = folders[PCB_num]

dataset_dir = f"/Users/mh/Downloads/datasets/{folder}"

In [14]:
parts = os.listdir(dataset_dir)

i = 0
for part in parts:
    print(i, end=" ")
    print(part)
    i += 1

0 screw2
1 connector1
2 supporter1
3 multiscrew2
4 connector2
5 screw1
6 supporter3
7 supporter2
8 multiscrew1


In [15]:
part_num = 0
PART_NAME = parts[part_num]

data_dir = f"{dataset_dir}/{PART_NAME}"
print(data_dir)

/Users/mh/Downloads/datasets/TM_Chassis_MULT/screw2


In [16]:
BATCH_SIZE = 32
IMAGE_SIZE = 224

TRAIN_DIR = f"{data_dir}/train"
VAL_DIR = f"{data_dir}/test"
TEST_DIR = f"{data_dir}/valid"

TRAIN_TOTAL = len(os.listdir(TRAIN_DIR + '/correct')) + len(os.listdir(TRAIN_DIR + '/incorrect'))
VAL_TOTAL = len(os.listdir(VAL_DIR + '/correct')) + len(os.listdir(VAL_DIR + '/incorrect'))
TEST_TOTAL = len(os.listdir(TEST_DIR + '/correct')) + len(os.listdir(TEST_DIR + '/incorrect'))

print(TRAIN_TOTAL, VAL_TOTAL, TEST_TOTAL)

432 168 168


In [17]:
trainIdg = ImageDataGenerator(
    rescale = 1.0 / 255,
    rotation_range = 1,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range = 0.2
)
trainGen = trainIdg.flow_from_directory(
    TRAIN_DIR,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    class_mode = 'binary',
    batch_size = BATCH_SIZE,
    shuffle = True
)

valIdg = ImageDataGenerator(rescale = 1.0 / 255)
valGen = valIdg.flow_from_directory(
    VAL_DIR,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    class_mode = 'binary',
    batch_size = BATCH_SIZE
)

testIdg = ImageDataGenerator(rescale = 1.0 / 255)
testGen = testIdg.flow_from_directory(
    TEST_DIR,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    class_mode = 'binary',
    batch_size = BATCH_SIZE
)

Found 432 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [18]:
baseModel = VGG16(
    include_top = False,
    weights = "imagenet",
    input_shape = (224, 224, 3)
)
baseModel.trainable = False

out = baseModel.get_layer('block5_pool').output  # the output of the base model
x = keras.layers.GlobalMaxPooling2D()(out)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(baseModel.input, x)  # the final CNN model

In [19]:
model.summary()

Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_37 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

In [20]:
print(f"<{part}>")

model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = [keras.metrics.BinaryAccuracy()]
)

history = model.fit(
    trainGen,
    epochs = 10,
    validation_data = valGen,
    steps_per_epoch = TRAIN_TOTAL // BATCH_SIZE,
    validation_steps = VAL_TOTAL // BATCH_SIZE,
    batch_size = BATCH_SIZE
)

if not os.path.exists(f"/Users/mh/Downloads/models/{folder}"):
  os.makedirs(f"/Users/mh/Downloads/models/{folder}")
model.save(f'/Users/mh/Downloads/models/{folder}/{PART_NAME}.h5')

print()
print(f"{part} saved..")

<multiscrew1>
Epoch 1/10
13/13 [==============================] - 32s 3s/step - loss: 0.3636 - binary_accuracy: 0.8675 - val_loss: 0.0788 - val_binary_accuracy: 1.0000
Epoch 2/10
13/13 [==============================] - 33s 3s/step - loss: 0.0422 - binary_accuracy: 1.0000 - val_loss: 0.0085 - val_binary_accuracy: 1.0000
Epoch 3/10
13/13 [==============================] - 33s 3s/step - loss: 0.0099 - binary_accuracy: 1.0000 - val_loss: 0.0035 - val_binary_accuracy: 1.0000
Epoch 4/10
13/13 [==============================] - 33s 3s/step - loss: 0.0039 - binary_accuracy: 1.0000 - val_loss: 0.0023 - val_binary_accuracy: 1.0000
Epoch 5/10
13/13 [==============================] - 33s 3s/step - loss: 0.0031 - binary_accuracy: 1.0000 - val_loss: 0.0016 - val_binary_accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 34s 3s/step - loss: 0.0024 - binary_accuracy: 1.0000 - val_loss: 0.0011 - val_binary_accuracy: 1.0000
Epoch 7/10
13/13 [==============================] - 33s 3s/ste

In [ ]:
test_loss, test_acc = model.evaluate(testGen, verbose=2)
print('\nTest accuracy: ', test_acc)

In [12]:
parts = os.listdir(dataset_dir)

i = 0
for part in parts:
    print(f"<{part}>")
    data_dir = f"{dataset_dir}/{part}"
    
    BATCH_SIZE = 32
    IMAGE_SIZE = 224

    TRAIN_DIR = f"{data_dir}/train"
    VAL_DIR = f"{data_dir}/test"
    TEST_DIR = f"{data_dir}/valid"
    
    TRAIN_TOTAL = len(os.listdir(TRAIN_DIR + '/correct')) + len(os.listdir(TRAIN_DIR + '/incorrect'))
    VAL_TOTAL = len(os.listdir(VAL_DIR + '/correct')) + len(os.listdir(VAL_DIR + '/incorrect'))
    TEST_TOTAL = len(os.listdir(TEST_DIR + '/correct')) + len(os.listdir(TEST_DIR + '/incorrect'))

    trainIdg = ImageDataGenerator(
        rescale = 1.0 / 255,
        rotation_range = 1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        shear_range = 0.2,
        zoom_range = 0.2
    )
    trainGen = trainIdg.flow_from_directory(
        TRAIN_DIR,
        target_size = (IMAGE_SIZE, IMAGE_SIZE),
        class_mode = 'binary',
        batch_size = BATCH_SIZE,
        shuffle = True
    )

    valIdg = ImageDataGenerator(rescale = 1.0 / 255)
    valGen = valIdg.flow_from_directory(
        VAL_DIR,
        target_size = (IMAGE_SIZE, IMAGE_SIZE),
        class_mode = 'binary',
        batch_size = BATCH_SIZE
    )

    testIdg = ImageDataGenerator(rescale = 1.0 / 255)
    testGen = testIdg.flow_from_directory(
        TEST_DIR,
        target_size = (IMAGE_SIZE, IMAGE_SIZE),
        class_mode = 'binary',
        batch_size = BATCH_SIZE
    )
    
    baseModel = VGG16(
        include_top = False,
        weights = "imagenet",
        input_shape = (224, 224, 3)
    )
    baseModel.trainable = False

    out = baseModel.get_layer('block5_pool').output  # the output of the base model
    x = keras.layers.GlobalMaxPooling2D()(out)
    x = keras.layers.Dense(512, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(baseModel.input, x)  # the final CNN model
    

    model.compile(
        optimizer = keras.optimizers.Adam(),
        loss = keras.losses.BinaryCrossentropy(from_logits=True),
        metrics = [keras.metrics.BinaryAccuracy()]
    )

    history = model.fit(
        trainGen,
        epochs = 10,
        validation_data = valGen,
        steps_per_epoch = TRAIN_TOTAL // BATCH_SIZE,
        validation_steps = VAL_TOTAL // BATCH_SIZE,
        batch_size = BATCH_SIZE
    )

    if not os.path.exists(f"/Users/mh/Downloads/models/VCG16/{folder}"):
      os.makedirs(f"/Users/mh/Downloads/models/VCG16/{folder}")
    model.save(f'/Users/mh/Downloads/models/VCG16/{folder}/{part}.h5')

    print()
    print(f"{part} saved..\n\n")

<screw2>
Found 432 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Epoch 1/10
13/13 [==============================] - 32s 3s/step - loss: 0.1966 - binary_accuracy: 0.9225 - val_loss: 0.0145 - val_binary_accuracy: 1.0000
Epoch 2/10
13/13 [==============================] - 33s 3s/step - loss: 0.0191 - binary_accuracy: 0.9975 - val_loss: 0.0024 - val_binary_accuracy: 1.0000
Epoch 3/10
13/13 [==============================] - 33s 3s/step - loss: 0.0050 - binary_accuracy: 1.0000 - val_loss: 9.0995e-04 - val_binary_accuracy: 1.0000
Epoch 4/10
13/13 [==============================] - 33s 3s/step - loss: 0.0039 - binary_accuracy: 1.0000 - val_loss: 8.0430e-04 - val_binary_accuracy: 1.0000
Epoch 5/10
13/13 [==============================] - 33s 3s/step - loss: 0.0030 - binary_accuracy: 1.0000 - val_loss: 6.8083e-04 - val_binary_accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 33s 3s/step - loss: 0.0047 - binar

Epoch 9/10
9/9 [==============================] - 23s 3s/step - loss: 0.0018 - binary_accuracy: 1.0000 - val_loss: 0.0018 - val_binary_accuracy: 1.0000
Epoch 10/10
9/9 [==============================] - 23s 3s/step - loss: 0.0015 - binary_accuracy: 1.0000 - val_loss: 0.0013 - val_binary_accuracy: 1.0000

connector2 saved..


<screw1>
Found 288 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Found 112 images belonging to 2 classes.
Epoch 1/10
9/9 [==============================] - 23s 3s/step - loss: 0.5517 - binary_accuracy: 0.6944 - val_loss: 0.3105 - val_binary_accuracy: 1.0000
Epoch 2/10
9/9 [==============================] - 23s 3s/step - loss: 0.2349 - binary_accuracy: 0.9271 - val_loss: 0.1247 - val_binary_accuracy: 1.0000
Epoch 3/10
9/9 [==============================] - 23s 3s/step - loss: 0.0760 - binary_accuracy: 1.0000 - val_loss: 0.0395 - val_binary_accuracy: 1.0000
Epoch 4/10
9/9 [==============================] - 23s 3s/step - loss: 0.0342 - binary